## Capacitated Vehicle Routing Problem

The capacitated vehicle routing problem (CVRP) is a combinatorial optimization problem in the area of logistics that aims to find a cheapest set of routes for a given fleet of vehicles such that all customers are served and the vehicle capacity is never exceeded.

The problem is formally defined as follows.
We are given
* a depot 0 where all vehicles are located in the beginning,
* a set of customers $C = \{1,\ldots,N\}$,
* a set of locations $L = \{0\} \cup C$),
* a demand $d_i \ge 0$ for each customer $i \in C$ (the depot has no demand, i.e., $d_0 = 0$),
* costs $c_{ij} \ge 0$ for traveling from customer or depot $i$ to customer or depot $j$,
* and a homogeneous fleet $K=\{1,\ldots,|K|\}$ of vehicles with load capacity $Q > 0$ which is the same for all vehicles.

We want to find
* an ordered visiting sequence of customers for each vehicle
* with minimal total routing costs,
such that
* each customer is visited exactly once,
* and the vehicle capacity is never exceeded.

This notebook demonstrates several ways to model the CVRP as mixed-integer linear program (MILP), without use of sophisticated column generation or cutting plane methods as needed in state-of-the-art approaches. 
Therefore, the presented models have limited performance compared to the state-of-the-art, independent of the used solver. 
Still, the formulations presented here can be a starting point to evaluate the performance for your problem instances.

### Instance Configuration

The first part is dedicated to setting up the instance data. 
For demonstration purposes, we generate artificial data in the following way:
* We predefine the number of customers, the number of available vehicles, their load capacity, and a range for the customer demands.
* We place the depot and customers randomly in a 1000x1000 grid. 
This allows us to compute Euclidian distances which lead to reasonable relations between different connections. 
These distances are interpreted as traveling costs.
* Customer demands are randomly generated in the predefined range.

In [5]:
# library

import math
import random
import gurobipy as gp
from gurobipy import GRB


In [6]:
### instance configuration

# Example1:  30,  5, 30, 2, 5
# Example2: 100, 60,  3, 1, 2
numCustomers = 30
maxNumVehicles = 5
vehicleCapacity = 30
demandMin = 2
demandMax = 5

### Instance Generation

In [7]:
# instance generation

random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)] # set of customers
locations = [depot] + customers # set of locations
connections = [(i, j) for i in locations for j in locations if i != j] # set of arcs
vehicles = [*range(1, maxNumVehicles + 1)] # set of vehicles

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]

# dictionary of Euclidean distance between each pair of points (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for i in locations
    for j in locations
    if i != j
}

# create random demands in range [demandMin,demandMax]
demands = {i: random.randint(demandMin, demandMax) for i in locations}

# depot has no demand
demands[0] = 0  

### Basic Route Model

First, we state a basic route model that uses binary variables $x_{ij}$ for each potential directed connection from location (depot or customer) $i$ to location $j$:
* The objective is to minimize the sum of costs of all used connections. 
* Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one nextCustomer connection.
* At most $|K|$ vehicles can be used.
* We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity. 
This constraint is not necessary but sometimes helps to improve performance. 
Basically, this is the capacity cut for set $S = C$, see below.

\begin{align*}
\min ~ & \sum_{i,j \in L} c_{ij} x_{ij} \\
& \sum_{i \in L} x_{ij} = 1, ~ \forall j \in C \\
& \sum_{j \in L} x_{ij} = 1, ~ \forall i \in C \\
& \sum_{j \in C} x_{0j} \le |K| \\
& \sum_{j \in C} x_{0j} \ge \left\lceil \frac{\sum_{i \in C} d_i}{Q} \right\rceil
\end{align*}

#### Optional: Vehicle-indexed Basic Route Model

Note that we did not use variables that assign connections or customers to particular vehicles. 
We have a homogeneous fleet in which all vehicles have exactly the same characteristics, so it does not matter which vehicle is assigned to which route. 
Using vehicle-specific variables could potentially simplify the solution retrieval afterwards, but would add many additional variables and constraints. 
Furthermore, solution symmetry would be introduced since each set of routes could be assigned to any permutation of vehicles, leading exactly to the same objective value but to different model solutions. 
This could significantly deteriorate solution performance.

For demonstration purposes, we provide the option to use vehicle-indexed binary route variables $v^k_{ij}$ for each vehicle $k \in K$, and each connection $(i,j)$ and the following basic route model:

\begin{align*}
\min ~ & \sum_{k \in K} \sum_{i,j \in L} c_{ij} v^k_{ij} \\
& \sum_{k \in K} \sum_{i \in L} v^k_{ij} = 1, ~ \forall j \in C \\
& \sum_{k \in K} \sum_{j \in L} v^k_{ij} = 1, ~ \forall i \in C \\
& \sum_{k \in K} \sum_{j \in C} v^k_{0j} \ge \left\lceil \frac{\sum_{i \in C} d_i}{Q} \right\rceil \\
& \sum_{i \in L} v^k_{ij} = \sum_{i \in L} v^k_{ji}, ~ \forall j \in C, ~ k \in K
\end{align*}

Note that the relation between $x$ and $v$ variables can be stated by:
$$\sum_{k \in K} v^k_{ij} = x_{ij}, ~ \forall i,j \in L, ~ i \neq j$$
Therefore, all constraints except for the last set of flow conservation constraints are already implied by the basic route model defined only on the $x$ variables.

In [8]:
### model configuration

# 0: no vehicle index, 1: vehicle-indexed route variables
vehicleIndexedRoutes = 0  

### Basic Route Model Implementation

In [9]:
def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

In [10]:
# formulation

# create model for CVRP instance
model = gp.Model("cvrp")

# binary variables x(i,j):
# x(i,j) is 1 if some vehicle is going from node i to node j, 0 otherwise
x = model.addVars(connections, vtype=GRB.BINARY, name="x")

# objective function: minimize sum of connection costs
model.setObjective(x.prod(costs), GRB.MINIMIZE)

# all customers have exactly one incoming and one outgoing connection
model.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

# vehicle limits
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")
model.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

if vehicleIndexedRoutes == 1:
    # binary variables v(i,j,k): is 1 if vehicle k is going from node i to node j, 0 otherwise
    v = model.addVars(connections, vehicles, vtype=GRB.BINARY, name="v")

    # relation to vehicle-independent route variables
    model.addConstrs(
        (v.sum(i, j, "*") == x[i,j] for (i,j) in connections), 
        name="linking"
        )

    # all customers have exactly one incoming and flow conservation has to hold for each vehicle
    model.addConstrs(
        (v.sum("*", j, k) == v.sum(j, "*", k) for j in customers for k in vehicles), 
        name="flowConservation"
        )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-15


### Load Models

Up to now we still have two issues in our model:
1. The vehicle capacity is ignored, so solutions might include routes with total customer demand that exceed the capacity.
2. Less obvious is the fact that in the base route model solutions might have sub-tours, i.e., routes that are disconnected from the depot and include only customers.

We will see that the following modeling approaches solve both issues at once.

#### Capacity Cut Model

The first approach only uses the binary connection variables we already have. 
We state the fact that for each subset of customers there is a minimum amount of vehicles required to serve them:

\begin{align*}
\sum_{i \notin S, j \in S} x_{ij} & \ge \left\lceil \frac{\sum_{c \in S} d_c}{Q} \right\rceil ~ \forall S \subseteq C
\end{align*}

These inequalities are called (rounded) capacity cuts and state that there has to be a minimum amount of connections going into each customer subset $S$. 
It can be shown that this is enough to ensure that the vehicle capacity is satisfied in all solutions that are feasible for these capacity cuts (together with the base route model). 

Additionally, the capacity cuts eliminate all sub-tours allowed in the base model. 
To see this, we define $S$ to be the set of customers that form a sub-tour, i.e., are not connected to the depot.
Then, we can easily see that the corresponding capacity cut is violated since no connection is going into set $S$ from outside.

The downside of the capacity cuts is that they are exponentially many so it is practically impossible to state all of them for a reasonable amount of customers. 
Usually, they are added dynamically in a cutting plane approach (with lazy-constraint and user-cut callbacks) but this involves according sophisticated callback methods to detect violated capacity cuts.

Nevertheless, we (optionally) add some of the capacity cuts to the model, i.e., those that correspond to customer sets $S$ with cardinality 2 or 3. 
They are not required for validity of the model when one of the following load model approaches are present but they usually help to improve dual bounds.

Below we use an equivalent form of the capacity cuts above that is much sparser for small customer sets and can be derived by exploiting the constraints from the base model.

\begin{align*}
\sum_{i,j \in S} x_{ij} & \le |S| - \left\lceil \frac{\sum_{c \in S} d_c}{Q} \right\rceil ~ \forall S \subseteq C
\end{align*}


#### Big-M Model

The following modeling approach is used very frequently since it is quite intuitive and also used in many other optimization models. 
The main idea is to associate a continuous variable $y_i \in [0,Q]$ to each customer $i \in C$ that represents the load of the vehicle after picking up the customer's demand. 
Note that it does not matter whether we pick up demands or deliver them, as long as we stay consistent for all customers the set of feasible solutions stays the same.

The following constraints model the fact that if connection $(i,j)$ is used then the vehicle load at $j$ is equal to the vehicle load at $i$ plus the demand at $j$. 
This if-then statement can be implemented with indicator constraints or directly with a Big-M approach:

\begin{align*}
y_0 & = 0 \\
y_i + d_j & \le y_j + Q (1 - x_{ij}) ~ \forall i \in L,~ j \in C, ~ i \neq j \\
y_i + d_j & \ge y_j - (Q - d_i - d_j) (1 - x_{ij}) ~ \forall i \in L, ~ j \in C, ~ i \neq j 
\end{align*}

Since we limit the load variables by the vehicle capacity, the latter cannot be exceeded.
Additionally, sub-tours are eliminated since the load has to monotonically increase along a route. 
In case of a sub-tour this would result in a conflict of the load values at some point. 
Contrary to this, in feasible routes the increase of load values is not required when coming back to the depot (note that the constraints above are not defined for $j = 0$).

Although this approach only increases the model size slightly, the corresponding dual bounds obtained by solving the continuous relaxation are often weak. 
This can lead to large branch-and-bound trees and poor solution performance.

#### Flow Model

Now, we extend the meaning of the variables used in the Big-M model. 
We introduce continuous variables $z_{ij} \in [0,Q]$ for each connection $(i,j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$. 
We can also express the new variables with the previous ones in a non-linear way, i.e., $z_{ij} = y_i \cdot x_{ij}$. 
In a different interpretation, these variables model the flow of goods on connection $(i,j)$ which suggests to use flow conservation constraints:

\begin{align*}
z_{0j} & = 0 ~ \forall j \in C \\
\sum_{i \in L} z_{ij} + d_j & = \sum_{i \in L} z_{ji} ~ \forall j \in C \\
z_{ij} & \ge d_i x_{ij} ~ \forall i \in C,~ j \in L, ~ i \neq j \\
z_{ij} & \le (Q - d_j) x_{ij} \; \forall i \in C, ~ j \in L, ~ i \neq j 
\end{align*}

As the last two sets of constraints state, there can only be non-zero flow on connections that are chosen in the solution.
The vehicle capacity is satisfied by definition, and with a similar argument as for the Big-M model sub-tours cannot appear in solutions since the flow conservation constraints (that are not defined for $j = 0$) do not allow a route including only customers.

Although there are more variables in this model, in most cases it pays off since it provides dual bounds at least as good as the ones of the Big-M model, and mostly better.


#### Multi-Commodity Flow Model

We can even go one step further and disaggregate the flows by customer. 
We introduce variables $f^c_{ij} \in [0,1]$ for each customer $c$ and connection $(i,j)$ to denote the fraction of customer $c$'s demand that has been picked up at customer $c$ and transported on connection $(i,j)$ back to the depot.
The resulting flow system can be stated as follows:

\begin{align*}
f^c_{0j} & = 0 ~ \forall c,j \in C \\
\sum_{i \in L} f^j_{ji} & = 1 ~ \forall j \in C \\
\sum_{i \in L} f^j_{i0} & = 1 ~ \forall j \in C \\
\sum_{i \in L} f^c_{ij} & = \sum_{i \in L} f^c_{ji} ~ \forall c,j \in C, ~ c \neq j \\
f^c_{ij} & \le x_{ij} ~ \forall c \in C, ~ i,j \in L, ~ i \neq j \\
\sum_{c \in C} d_c f^c_{ij} & \le Q x_{ij} ~ \forall i,j \in L, ~ i \neq j
\end{align*}

It can be shown that the resulting LP bound is at least as good as the one from the previous flow model, often much better. 
The larger number of variables and constraints, however, usually leads to worse performance in practice.

#### Discretized Model

If the set of achievable vehicle load values is rather small then it makes sense to consider the modeling concept of discretization. 
For simplicity, we assume that the vehicle capacity and all demands are integer values, and therefore the load of the vehicle can only be in the set $D=\{0,1,...,Q\}$. 
Continuous values could be scaled appropriately but note that set $D$ should not get too large.

Next, we introduce binary variables $w_{ijd}$ to state whether connection $(i,j)$ is used with vehicle load $d \in D$ or not.
The set of constraints looks similar to the flow constraints in the base route model:

\begin{align*}
w_{0jd} & = 0 ~ \forall j \in C, ~ d \neq 0 \\
\sum_{i \in L} w_{ijd} & = \sum_{i \in L} w_{ji,d+d_j} ~ \forall j \in C, ~ d \in D \\
x_{ij} & = \sum_{d \in D} w_{ijd} ~ \forall i,j \in L, ~ i \neq j
\end{align*}

As you can see, the number of variables and constraints strongly depends on the size of set $D$.
By definition of the variables, there cannot be a vehicle load larger than $Q$, implicitely ensuring the vehicle capacity.
Again, since the vehicle load is monotonically increasing there would be a constraint conflict in a sub-tour only consisting of customers.

The benefit of this modeling approach is that the model structure is quite simple, there are only binary variables, equalities, and coefficients of 0 and 1. 
This can lead to better performance in some cases.

Note that the modeling concepts discussed here can also be used for other constraints arising in vehicle routing, e.g., time windows at customers, time or distance constraints for routes, etc.

In the following implementation, you can choose which load model you want to use, and whether you want to additionally add capacity cuts or not. 
This allows us to do some performance comparisons.

In [11]:
### MODEL CONFIGURATION

capacityCuts = 0  # 0: off, 2: cuts for |S|=2, 3: cuts for |S|<=3
loadModelType = 2  # 1: big-M, 2: flow, 3: multi-flows, 4: discretization

### Load Model Implementation

In [12]:
def addRoundedCapacityCuts():
    if capacityCuts >= 2:
        model.addConstrs(
            (
                x[i, j] + x[j, i] <= 2 - numVehiclesNeededForCustomers([i, j])
                for i in customers
                for j in range(i + 1, numCustomers + 1)
            ),
            name="capcut2",
        )
    if capacityCuts >= 3:
        model.addConstrs(
            (
                x[i, j] + x[j, i] + x[i, k] + x[k, i] + x[j, k] + x[k, j]
                <= 3 - numVehiclesNeededForCustomers([i, j, k])
                for i in customers
                for j in range(i + 1, numCustomers + 1)
                for k in range(j + 1, numCustomers + 1)
            ),
            name="capcut3",
        )

In [13]:
def addVehicleLoadConstraintsByBigM():

    y = model.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
    y[0].UB = 0  # empty load at depot

    model.addConstrs(
        (
            y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM1",
    )
    model.addConstrs(
        (
            y[i] + demands[j]
            >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="loadBigM2",
    )

In [14]:
def addVehicleLoadConstraintsByFlows():

    z = model.addVars(connections, lb=0, ub=vehicleCapacity, name="z")

    for i in customers:
        z[0, i].UB = 0

    model.addConstrs(
        (z.sum("*", j) + demands[j] == z.sum(j, "*") for j in customers),
        name="flowConservation",
    )
    model.addConstrs(
        (
            z[i, j] >= demands[i] * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadLowerBound",
    )
    model.addConstrs(
        (
            z[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
            for i in customers
            for j in locations
            if i != j
        ),
        name="loadUpperBound",
    )

In [15]:
def addVehicleLoadConstraintsByMultiCommodityFlows():

    f = model.addVars(connections, customers, lb=0, ub=1, name="f")

    for c in customers:
        for (i, j) in connections:
            if i == 0 or c == j:
                f[i, j, c].UB = 0

    model.addConstrs((f.sum(j, "*", j) == 1 for j in customers), name="pickup")
    model.addConstrs((f.sum("*", 0, j) == 1 for j in customers), name="return")
    model.addConstrs(
        (
            f.sum("*", j, c) == f.sum(j, "*", c)
            for j in customers
            for c in customers
            if c != j
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (f[i, j, c] <= x[i, j] for (i, j) in connections for c in customers),
        name="links",
    )
    model.addConstrs(
        (
            gp.quicksum(demands[c] * f[i, j, c] for c in customers)
            <= (vehicleCapacity - demands[j]) * x[i, j]
            for (i, j) in connections
        ),
        name="capacity",
    )

In [16]:
def addVehicleLoadConstraintsByDiscretization():

    loadValues = [*range(vehicleCapacity + 1)]
    w = model.addVars(connections, loadValues, vtype=GRB.BINARY, name="w")

    for j in customers:
        for d in loadValues:
            if d != 0:
                w[0, j, d].UB = 0

    model.addConstrs(
        (
            w.sum("*", j, d) == w.sum(j, "*", d + demands[j])
            for j in customers
            for d in loadValues
        ),
        name="flowConservation",
    )
    model.addConstrs(
        (x[i, j] == w.sum(i, j, "*") for (i, j) in connections), name="linking"
    )

In [17]:
if capacityCuts > 0:
    addRoundedCapacityCuts()

if loadModelType == 1:
    addVehicleLoadConstraintsByBigM()
elif loadModelType == 2:
    addVehicleLoadConstraintsByFlows()
elif loadModelType == 3:
    addVehicleLoadConstraintsByMultiCommodityFlows()
elif loadModelType == 4:
    addVehicleLoadConstraintsByDiscretization()

### Solve Model

In [18]:
# parameters value
MAX_CPU_TIME = 600
EPSILON= 1.e-6

# export .lp
#model.write(file_name+"_model.lp")

# parameters config
#model.Params.IterationLimit = 1000 # number of simplex iteration
#model.Params.TimeLimit = 60 # time limite
model.Params.MIPGap = 1.e-6 # MIPGap
#model.Params.method = 1 # method root
#model.Params.NodeMethod = -1 #  -1=automatic, 0=primal simplex, 1=dual simplex, and 2=barrier
model.Params.Threads = 1
model.Params.Presolve = 0
model.Params.Cuts = 1
    
# Turn off display
gp.setParam('OutputFlag', 0)

# Turn off heuristics
#gp.setParam('Heuristics', 0)

# open log file
#_ = open('log/cvrp.log', 'w')

model.optimize()

Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Set parameter Presolve to value 0
Set parameter Cuts to value 1


### Solution Retrieval

What we basically want to know is the ordered sequence of customers each vehicle should visit.
We did not use variables that assign connections or customers to vehicles (when using the basic route model without vehicle-index), so the question is whether it is possible to retrieve a unique set of routes only based on the set of used connections defined by the $x$-variables.
This is indeed possible since each customer is visited exactly once. We iteratively pick one of the connections leaving the depot, and follow the path until we get back to the depot. Note that there is exactly one incoming and one outgoing connection at each customer, so it is clear where a route continues.

Additional to reconstructing the routes, we check whether the vehicle load does not exceed the capacity on each route, and each customer is visited from the depot. This is not necessary as long as the model is correct but it helps significantly in the debugging phase.

In [30]:
status = 0
if model.status == GRB.OPTIMAL:
    status = 1
 
objbound = model.objBound
objval = model.objVal
mipgap = model.MIPGap
runtime = model.Runtime
nodecount = model.NodeCount

In [31]:
if model.SolCount >= 1:

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one 
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0, load = {vehicleLoad} of {vehicleCapacity}")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")

Solution contains 4 routes:
Route 0: 0 -> 10 -> 27 -> 18 -> 0, load = 11 of 30
Route 1: 0 -> 16 -> 5 -> 7 -> 3 -> 15 -> 23 -> 4 -> 29 -> 8 -> 0, load = 30 of 30
Route 2: 0 -> 22 -> 25 -> 24 -> 12 -> 19 -> 2 -> 11 -> 20 -> 13 -> 0, load = 29 of 30
Route 3: 0 -> 30 -> 17 -> 26 -> 14 -> 9 -> 1 -> 6 -> 21 -> 28 -> 0, load = 30 of 30
Unvisited customers: 

In [33]:
print("bound =", objbound)
print("val =",objval)
print("gap =", mipgap)
print("time = ", runtime)
print("nodes = ", nodecount)
print("status = ", status)

bound = 6046.999999999999
val = 6046.999999999999
gap = 0.0
time =  143.54062795639038
nodes =  46009.0
status =  1


In [ ]:
arquivo = open(f'result/cvrp.csv','a')
arquivo.write(
#    str(f"{dim}_{perc}_{t}")+';'
    +str(round(objbound,1))+';'
    +str(round(objval,1))+';'
    +str(round(mipgap,2))+';'
    +str(round(runtime,2))+';'
    +str(round(nodecount,1))+';'
    +str(round(status,1))+'\n'
)
arquivo.close()

In [15]:
if model.Status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("iis.ilp")

In [16]:
# free resources
model.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment
